<a href="https://colab.research.google.com/github/m4a1carbin4/LightKorLLM/blob/dev/colab_script/LightKorLLM_auto_gptq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#torch , cuda , cudnn version check

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:2.0.1+cu118
cuda version: 11.8
cudnn version:8700


In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ # install auto-gptq with cuda 11.8

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4

In [ ]:
import json
import random
import time
from argparse import ArgumentParser

import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from datasets import Dataset
from transformers import AutoTokenizer

In [ ]:
def load_data(data_path, tokenizer, n_samples):
    with open(data_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    raw_data = random.sample(raw_data, k=min(n_samples, len(raw_data)))

    def dummy_gen():
        return raw_data

    def tokenize(examples):
        instructions = examples["instruction"]
        inputs = examples["input"]
        outputs = examples["output"]

        prompts = []
        texts = []
        for istr, inp, opt in zip(instructions, inputs, outputs):
            if inp:
                prompt = f"Instruction:\n{istr}\nInput:\n{inp}\nOutput:\n"
                text = prompt + opt
            else:
                prompt = f"Instruction:\n{istr}\nOutput:\n"
                text = prompt + opt
            prompts.append(prompt)
            texts.append(text)

        return {
            "texts": texts
        }

    dataset = Dataset.from_generator(dummy_gen)

    dataset = dataset.map(
        tokenize,
        batched=True,
        batch_size=len(dataset),
        num_proc=1,
        keep_in_memory=True,
        load_from_cache_file=False
    )

    dataset = dataset.to_list()

    return dataset

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!wget https://raw.githubusercontent.com/Beomi/KoAlpaca/main/ko_alpaca_data.json

--2023-09-30 13:11:04--  https://raw.githubusercontent.com/Beomi/KoAlpaca/main/ko_alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16389582 (16M) [text/plain]
Saving to: ‘ko_alpaca_data.json’

ko_alpaca_data.json 100%[===================>]  15.63M  --.-KB/s    in 0.04s   

2023-09-30 13:11:06 (385 MB/s) - ‘ko_alpaca_data.json’ saved [16389582/16389582]



In [ ]:
max_memory = dict()

max_memory = None

pretrained_model_dir = "heegyu/llama-2-ko-7b-chat"

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_dir,
    #use_fast=args.fast_tokenizer,
    trust_remote_code=False
)

model = AutoGPTQForCausalLM.from_pretrained(
    pretrained_model_dir,
    quantize_config=BaseQuantizeConfig(bits=4, group_size=128, desc_act=False,model_file_base_name="model"),
    max_memory=max_memory,
    trust_remote_code=False
)

examples = load_data("ko_alpaca_data.json", tokenizer, 2048)
examples_for_quant = [
    tokenizer(example["texts"])
    for example in examples
]

#examples_for_quant

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2048 [00:00<?, ? examples/s]

In [ ]:
start = time.time()
model.quantize(
    examples_for_quant,
    batch_size=1,
    use_triton=True,
    autotune_warmup_after_quantized=True
)
end = time.time()
print(f"quantization took: {end - start: .4f}s")

100%|██████████| 12/12 [00:40<00:00,  3.39s/it]


quantization took:  3494.1061s


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
repo_id = "llama-2-7b-ko-auto-gptq-full-v2"
commit_message = f"AutoGPTQ model for llama-2-7b-ko-full: 4 bits"
model.push_to_hub(repo_id, save_dir="llama-2-7b-ko-auto-gptq-full-v2", commit_message=commit_message, use_auth_token=True)
tokenizer.push_to_hub(repo_id, save_dir="llama-2-7b-ko-auto-gptq-full-v2", commit_message=commit_message, use_auth_token=True)

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/WGNW/llama-2-7b-ko-auto-gptq-full-v2/commit/578d55428deb19ac07265ed01b81ff607de4255e', commit_message='AutoGPTQ model for llama-2-7b-ko-full: 4 bits', commit_description='', oid='578d55428deb19ac07265ed01b81ff607de4255e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
def gen(x,model,tokenizer):
    gened = model.generate(
        **tokenizer(
            x,
            return_tensors='pt',
            return_token_type_ids=False
        ).to("cuda"),
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

base_model = "WGNW/llama-2-7b-ko-auto-gptq-full-v2"
#peft_model = "WGNW/Llama-2-ko-7b-Chat-auto-gptq-4bit-peft-remon"

#config = PeftConfig.from_pretrained(peft_model)

model = AutoModelForCausalLM.from_pretrained(base_model,device_map="cuda")
#model = PeftModel.from_pretrained(model, peft_model)

tokenizer = AutoTokenizer.from_pretrained(base_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
gen("""당신은 AI 챗봇입니다. 사용자에게 도움이 되고 유익한 내용을 제공해야합니다. 답변은 길고 자세하며 친절한 설명을 덧붙여서 작성하세요.

### 사용자:
역대 미국 대통령에 대해 레포트 형식으로 작성해줘!

### 챗봇:""",model.to("cuda"),tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<s>당신은 AI챗봇입니다.사용자에게도움이되고유익한내용을제공해야합니다.답변은길고자세하며친절한설명을덧붙여서작성하세요.

###사용자:
역대미국대통령에대해레포트형식으로작성해줘!

###챗봇:
 미국역대대통령의 사례를 중심으로 사용자와 이야기하고 싶습니다.

1. 조지 워싱턴:
조지 워싱턴은 미국의 최초의 대통령으로, 자유와 독립된 국민에 기반한 국가의 잠재력을 발휘했습니다. 그는 미국의 독립전쟁을 이끄는 과정에서 독립을 선언하고 헌법에 기초한 헌법을 구축했습니다.

2. 토마스 제퍼슨:
토마스 제퍼슨은 미국의 5대 대통령과 7번째 대통령으로 임명되었으며, 미국에서 가장 위대한 정치가 중 하나로 평가됩니다. 그는 대서양 바다에서 가장 중요한 항해인 에드워드 호를 만들면서 미국을 해양강국으로 만들었습니다.

3. 토머스 제퍼슨:
제퍼슨은 미국의 6대와 7대 대통령으로 임명된 유일한 대통령입니다. 그는 미국의 산업혁명을 이끌었고, 미국에서 가장 영향력 있는 인물을 만듭니다. 그의 혁신적인 기업가 정신과 기술 분야에서 미국 경제의 발전에 기여했습니다.

4. 벤자민게이츠:
벤자민 게이브아츠는 미국의 12대와 20대 대통령을 역임하였습니다. 그는 대서양 북서부의 국가 안보를 위해 미국 혁명에서 맹활약했으며, 헌법과 미국을 동요한 사회정치를 구축하였습니다.

5. 마틴 루소:
마틴 루소는 미국의 7대와 8대 대통령으로 임명되어 미국 인권과 정치적 자유를 보호했습니다. 그는 혁명을 통해 평등을 지향하는 사회를 구축하고, 시민의 권리와 자치권을 강화하였습니다.

6. 에이브러햄 로저스 클린턴:
에이브러햄 로저스 클린턴은 미국의 22대와 26대 대통령으로 임명되었으며, 외교와 경제에서 미국 중심의 입장을 강조하였습니다. 그는 미국의 대외 관계와 경제 분야에서 놀라운 성과를 이루었고, 미국의 영향력을 높였습니다.

7. 존 F. 케네스 클린턴:
존 F. 케네스 클린턴은 미국에서의 첫 번째 여성 대통령입니다. 그녀는 외교와 경제 발전을 통해 미국 경제를 발전시켰고, 미국의 독립성을 